In [ ]:
!pip install torch==2.3.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -U transformers==4.46.1 datasets accelerate evaluate sentencepiece


Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.9/780.9 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 145.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 983.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1

In [ ]:
from google.colab import files
uploaded = files.upload()  # train.json 선택


Saving train_final.json to train_final.json


In [ ]:
!rm -rf ./output  # 이전 모델 완전히 제거

from datasets import load_dataset
from transformers import AutoTokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import torch

# 1️⃣ 데이터 로드
dataset = load_dataset("json", data_files="train_final.json")

# 2️⃣ 한국어 T5 로드 (paust/pko-t5-small)
tokenizer = AutoTokenizer.from_pretrained("paust/pko-t5-small")
model = T5ForConditionalGeneration.from_pretrained("paust/pko-t5-small")

# 3️⃣ 전처리
def preprocess(batch):
    model_inputs = tokenizer(batch["input"], truncation=True, padding="max_length", max_length=64)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["label"], truncation=True, padding="max_length", max_length=64)
    labels_ids = labels["input_ids"]
    labels_ids = [[(token if token != tokenizer.pad_token_id else -100) for token in l] for l in labels_ids]
    model_inputs["labels"] = labels_ids
    return model_inputs

tokenized = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)
tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# 4️⃣ 학습 설정
args = Seq2SeqTrainingArguments(
    output_dir="./output",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    num_train_epochs=5,
    fp16=True,
    logging_steps=100,
    save_strategy="epoch",
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
)

trainer.train()
import shutil, os

base = "./output"
for d in os.listdir(base):
    if d.startswith("checkpoint-"):
        shutil.rmtree(os.path.join(base, d))
print("✅ 중간 체크포인트 제거 완료!")

# ✅ 2️⃣ 그 다음에 모델 저장
trainer.save_model("./output")
tokenizer.save_pretrained("./output")
print("✅ 최종 모델 저장 완료!")


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/8614 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/tmp/ipython-input-2783279715.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
100,1.296700
200,0.149600
300,0.065600
400,0.037000
500,0.029600
600,0.024900
700,0.018900
800,0.017600
900,0.017600
1000,0.016800


✅ 중간 체크포인트 제거 완료!
✅ 최종 모델 저장 완료!


In [ ]:
import os

def get_folder_size(path):
    total = 0
    for root, dirs, files in os.walk(path):
        for f in files:
            fp = os.path.join(root, f)
            total += os.path.getsize(fp)
    return total

path = "/content/drive/MyDrive/my_model"
size_mb = get_folder_size(path) / (1024 * 1024)
print(f"📦 모델 폴더 크기: {size_mb:.2f} MB")


📦 모델 폴더 크기: 369.01 MB


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
model.save_pretrained("/content/drive/MyDrive/my_model")
tokenizer.save_pretrained("/content/drive/MyDrive/my_model")

('/content/drive/MyDrive/my_model/tokenizer_config.json',
 '/content/drive/MyDrive/my_model/special_tokens_map.json',
 '/content/drive/MyDrive/my_model/tokenizer.json')

In [ ]:
#내 컴퓨터로 직접 다운로드
from google.colab import files
import shutil

# 1️⃣ 모델 폴더 압축
shutil.make_archive("model_backup", "zip", "./output")

# 2️⃣ 내 컴퓨터로 다운로드
files.download("model_backup.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#🧩 2️⃣ Google Drive에 백업 (Colab 런타임 종료 대비)
#🧩 1️⃣ 내 컴퓨터로 직접 다운로드
from google.colab import files
import shutil

# 1️⃣ 모델 폴더 압축
shutil.make_archive("model_backup", "zip", "./output")

# 2️⃣ 내 컴퓨터로 다운로드
files.download("model_backup.zip")


In [ ]:
#나중에 다시 불러오기
from transformers import AutoTokenizer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/paust_t5_corrector")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/paust_t5_corrector")

print("✅ 모델 다시 불러오기 완료!")


모델 가져와서 다시 테스트하기

In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch

# 🔹 모델 경로 지정 (Drive에 저장했다면 아래 경로로 수정)
model_path = "/content/drive/MyDrive/paust_t5_corrector"

# 모델과 토크나이저 불러오기
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# GPU 사용 설정 (있으면)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("✅ 모델 로드 완료!")


In [ ]:
# 문장 교정 테스트 함수
def correct_sentence(text):
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=64)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return result.strip()


In [ ]:
#테스트 실행 예시
test_sentences = [
    "간다 오늘 집에",
    "공부 해야 나 내일",
    "밥 먹자 같이 오늘",
    "친구 만나 영화 봤다 같이",
    "학교 가 버스 타고 나는"
]

print("✅ 문맥 교정 테스트 결과\n")
for s in test_sentences:
    print(f"입력: {s}")
    print(f"출력: {correct_sentence(s)}\n")


모델평가

In [ ]:
!pip install sacrebleu rouge-score


  Using cached sacrebleu-2.5.1-py3-none-any.whl.metadata (51 kB)
  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.6 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=c3f0319803937c99409279a3259b27fd0e407a3376c17c96597e9a79677e27d0
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [ ]:
!pip install sacrebleu rouge-score chrf


ERROR: Could not find a version that satisfies the requirement chrf (from versions: none)
ERROR: No matching distribution found for chrf


In [ ]:
!pip install sacrebleu rouge-score

from datasets import load_dataset
from transformers import AutoTokenizer, T5ForConditionalGeneration
import evaluate
from tqdm import tqdm
import torch

# 1️⃣ 모델 불러오기
model = T5ForConditionalGeneration.from_pretrained("./output")
tokenizer = AutoTokenizer.from_pretrained("./output")
model.eval()

# 2️⃣ 데이터 로드
dataset = load_dataset("json", data_files="train_clean.json")["train"]

# 3️⃣ metric 불러오기
bleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
chrf = evaluate.load("chrf")

# 4️⃣ 평가 수행
preds, refs = [], []
for item in tqdm(dataset.select(range(200))):  # 200개 샘플
    text = item["input"]
    label = item["label"]
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=64)
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    preds.append(pred)
    refs.append(label)

# 5️⃣ 점수 계산
bleu_score = bleu.compute(predictions=preds, references=[[r] for r in refs])["score"]
chrf_score = chrf.compute(predictions=preds, references=refs)["score"]

# 6️⃣ 출력
print("📊 모델 성능 평가 결과 (200개 샘플 기준)")
print(f"✅ BLEU: {bleu_score:.2f}")
print(f"✅ chrF++: {chrf_score:.2f}")


100%|██████████| 200/200 [01:48<00:00,  1.85it/s]

📊 모델 성능 평가 결과 (200개 샘플 기준)
✅ BLEU: 79.50
✅ chrF++: 82.97


모델 직접 테스트

In [ ]:
!pip install gradio

import gradio as gr
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch

model = T5ForConditionalGeneration.from_pretrained("./output")
tokenizer = AutoTokenizer.from_pretrained("./output")

def correct_sentence(text):
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=64,
            num_beams=5,
            repetition_penalty=2.0,
            temperature=0.7,
            length_penalty=1.0,
            early_stopping=True
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

demo = gr.Interface(
    fn=correct_sentence,
    inputs="text",
    outputs="text",
    title="🧠 한국어 문맥 교정기 (T5-small)",
    description="비문을 자연스러운 문장으로 바꿔줍니다.",
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b599f7e7d66ab9694b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
